# Finjustere språkmodeller på Nav.no

I denne notatboken skal vi illustrere hvordan man kan komme i gang med
finjustering av språkmodeller. Vi skal gå gjennom stegene for å gjøre klart et
datasett basert på data på [datamarkedsplassen](https://data.ansatt.nav.no/).
Hvordan teste ytelsen på en embeddingmodell på dette datasettet. Og tilslutt,
skal vi se hvordan vi kan forbedre ytelsen med finjustering.

## Prosjektoppsett

Vi anbefaler at man bruker [`uv`](https://docs.astral.sh/uv/) for å opprette
prosjekt og styre avhengigheter.

La oss starte med å lage et prosjekt:

```bash
uv init --app --python 3.12 navno_finetune
```

Inne i prosjektet kan vi fjerne `main.py` (eller `hello.py` avhengig av din
versjon av `uv`). Hvis du ønsker å følge denne veiledningen kan man enkelt
opprettet en Jupyter Notebook fil og klippe og lime kode fra veiledningen.
Alternativt kan man strukturere kode etter eget ønske og bruke veiledningen til
inspirasjon.

::: {.callout collapse="true"}
## Gjenbruke _denne_ veiledningen

Denne veiledningen er også mulig å gjenbruke ordrett. For å være enkelt å
publisere er den strukturert som et Quarto prosjekt, men det er fullt mulig å
klone veiledningen og bare kjøre Quarto filen direkte.
:::

## Datasett

Før vi kan starte å finjustere trenger vi et datasett vi kan teste på og som vi
kan bruke til trening. Vi kommer til å benytte [innholdet på
Nav.no](https://data.ansatt.nav.no/dataproduct/6c7327e2-5894-4423-b6b2-52affa3f5b29/Innhold%20p%C3%A5%20Nav.no/7993897c-9fd4-46ee-86dd-5001621a2695)
som utgangspunkt.

### Laste ned rådata

La oss starte med å laste ned rådata fra BigQuery. For å gjøre dette kommer vi
til å bruke `google-cloud-bigquery` og [Polars](https://docs.pola.rs/).

::: {.callout-note}
## Nødvendige avhengigheter

For å installere avhengigheter kjører vi følgende `uv` kommandoer:

```bash
uv add google-cloud-bigquery
uv add polars --extra pyarrow
```
:::

Vi starter med å hente all rådata og opprette en Polars `DataFrame`.

In [2]:
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

# Bygg opp spørring og hent all data for gitt tidspunkt
QUERY = (
    'SELECT * FROM `nks-aiautomatisering-prod-194a.navno_crawl.navno` '
    'WHERE DATE(crawl_date) = DATE(2025, 02, 25)')
query_job = client.query(QUERY)
rows = query_job.result()  # Vent på nedlasting

df = pl.from_arrow(rows.to_arrow())  # Opprett dataframe med rådata

/Users/jorgen/Projects/navno_finetune/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


La oss inspisere dataene, før vi konverterer det til et mer passende format for
språkmodeller.

In [ ]:
#| column: page
df.head()

path,id,type,created,last_modified,crawl_date,language,anchor,taxonomy,area,owner,display_name,ingress,headers,content
str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,list[str],list[str],list[str],str,str,list[str],str
"""/afp-etteroppgjoret""","""3f7ccc77-148b-4c1b-9f03-ddcc27…","""no.nav.navno:current-topic-pag…",2023-09-05 12:15:10.338 UTC,2023-09-05 12:41:40.281943 UTC,2025-02-25 12:40:51.156586 UTC,"""no""",null,[],"[""pension""]","[""familie_og_pensjonsytelser_styringsenhet""]","""Avtalefestet pensjon (AFP) og …","""""",[],"""<h4><strong>Har du fått brev o…"
"""/protese-ortose-ortopediskesko…","""3c26542b-cbac-4b7b-8266-4a7202…","""no.nav.navno:situation-page""",2022-06-20 11:49:32.143 UTC,2023-11-01 08:06:28.620071 UTC,2025-02-25 12:40:51.156586 UTC,"""no""","""reiseutgifter""",[],"[""accessibility""]","[""hjelpemidler_og_tilrettelegging""]","""Trenger en protese, ortose, or…","""Om ortopediske hjelpemidler, o…","[""<h2>Reiseutgifter</h2>""]","""<p>Du kan få dekket reiseutgif…"
"""/protese-ortose-ortopediskesko…","""3c26542b-cbac-4b7b-8266-4a7202…","""no.nav.navno:situation-page""",2022-06-20 11:49:32.143 UTC,2023-11-01 08:06:28.620071 UTC,2025-02-25 12:40:51.156586 UTC,"""no""","""grunnstonad""",[],"[""accessibility""]","[""hjelpemidler_og_tilrettelegging""]","""Trenger en protese, ortose, or…","""Om ortopediske hjelpemidler, o…","[""<h2>Grunnstønad</h2>""]","""<p>Du eller barnet ditt kan ha…"
"""/protese-ortose-ortopediskesko…","""3c26542b-cbac-4b7b-8266-4a7202…","""no.nav.navno:situation-page""",2022-06-20 11:49:32.143 UTC,2023-11-01 08:06:28.620071 UTC,2025-02-25 12:40:51.156586 UTC,"""no""","""aktuelle-produktsider""",[],"[""accessibility""]","[""hjelpemidler_og_tilrettelegging""]","""Trenger en protese, ortose, or…","""Om ortopediske hjelpemidler, o…","[""<h2>Aktuelle ordninger</h2>""]","""<div><a href=""/ortopediskesko""…"
"""/protese-ortose-ortopediskesko…","""3c26542b-cbac-4b7b-8266-4a7202…","""no.nav.navno:situation-page""",2022-06-20 11:49:32.143 UTC,2023-11-01 08:06:28.620071 UTC,2025-02-25 12:40:51.156586 UTC,"""no""","""andre-som-kan-hjelpe""",[],"[""accessibility""]","[""hjelpemidler_og_tilrettelegging""]","""Trenger en protese, ortose, or…","""Om ortopediske hjelpemidler, o…","[""<h2>Andre som kan hjelpe</h2>""]","""<div><a href=""https://www.hjel…"


### Strukturere data for språkmodeller

For å finjustere en embeddingmodell er det i hovedsak _fire_ ulike måter å
strukturere et datasett:

- **Positive pair**: Et par setninger som er relatert (f.eks `(spørsmål,
svar)`).
- **Triplets**: Likt som _positive pair_, men med et anti-relatert element.
    - Fordi vi kan bruke treningsfunksjoner (_loss_-funksjon) som kan gjennbruke
    data i _positiv pair_ datasettet er ikke dette formatet like mye brukt.
- **Pair with Similarity Score**: Et par setninger og en verdi som representerer
hvor like disse setningene er.
- **Text with Classes**: En setning med tilhørende klasse. Kan konverteres til
andre formater over.

::: {.column-margin}
Hentet fra [SBERT.net - Dataset
Overview](https://sbert.net/docs/sentence_transformer/dataset_overview.html).
:::

Basert på dataene over så er det naturlig å velge _Positiv Pair_. Dette er fordi
vi kan koble flere kolonner sammen for å lage disse parene. Vi kan for eksempel
koble tittel og innhold sammen, noe som burde forsterke koblingen mellom tittel
og relevant innhold for språkmodellen.

La oss starte med det åpenbare `(tittel, innhold)` paret

In [ ]:
df_title_content = df.select(
    pl.col("display_name").alias("anchor"),
    pl.col("content").alias("positiv")
)
df_title_content.head()

En annen åpenbar kobling er `(tittel, ingress)`

In [ ]:
df_title_ingres = df.filter(
    pl.col("ingress").str.len_bytes() > 0).select(
        pl.col("display_name").alias("anchor"),
        pl.col("ingress").alias("positiv")
    ).unique()
df_title_ingres.head()

La oss også koble side tittel sammen med titler til innholdet og innhold

In [ ]:
df_all_titles_content = df.filter(pl.col("headers").list.len() > 0).select(
    (pl.col("display_name") + "\n" + pl.col("headers").list.join(separator="\n")).alias("anchor"),
    pl.col("content").alias("positiv")
)
df_all_titles_content.head()

La oss koble alle disse tabellene sammen og for å lage et endelig datasett.

In [ ]:
df_train = pl.concat([df_title_content, df_title_ingres, df_all_titles_content])
df_train.describe()

### Rense datasett

Før vi sier oss fornøyd skal vi vaske dataene våre litt. I tabellene over har du
kanskje lagt merke til at flere av kolonnene inneholder HTML elementer. Det er
ikke i utgangspunktet noe galt å bruke dette for trening, men siden vi her
fokuserer på en embeddingmodell ønsker vi at den fokuserer på semantikken og
ikke formatet. Vi skal derfor prøve å renske bort alle HTML tag-er^[Vi gjør det
enkelt med en `regex` inspirert av
[StackOverflow](https://stackoverflow.com/a/12982689).].

In [ ]:
df_train = df_train.select(
    pl.col("anchor").str.replace_all("<.*?>", " ").str.strip_chars(),
    pl.col("positiv").str.replace_all("<.*?>", " ").str.strip_chars(),
)
df_train.head()

### Lagre til fil

La oss avslutte med å lagre data til en fil slik at vi enkelt kan gjenskape
treningen og samtidig dele data med andre på en enkel måte. Et format som kan
være praktisk er [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) som
både er effektivt for å lagre dataframe data og samtidig er godt støttet i de
fleste verktøy vi bruker i Nav.

In [ ]:
df_train.write_parquet("dataset.parquet")

## Trening og test

Nå som vi har laget et fullstendig treningssett kan vi begynne å tenke på å dele
opp i en trenings del og en test del. Dette gjør vi for å ha en del som modellen
får lov til å se på, trenings delen, og en del som er helt ny for modellen, test
delen. Ved å skille slik får vi mulighet til å evaluere hvor godt modellen
fungerer på ting den ikke har sett før.

Vi starter med å legge til en `ID` kolonne på datasettet vårt slik at vi kan
unikt identifisere rader, dette kommer vi til å trenge senere.

In [ ]:
dataset = pl.read_parquet("dataset.parquet").with_row_index("id")

Deretter deler vi datasettet i en del for trening, meste parten, og en del for
testing.

In [ ]:
# Litt komplisert å lage trening/test split i Polars
#
# Vi starter med å randomisere hele datasettet
dataset = dataset.sample(fraction=1, shuffle=True, seed=12345)
# Beregne antall rader vi skal bruke
num_test = int(0.2 * len(dataset))
# Deretter ta de første `num_test` radene til test
test_dataset = dataset.head(num_test)
# Tilslutt tar vi alle utenom de første `num_test` radene til trening
train_dataset = dataset.tail(-num_test)

### Corpus

Nå som vi har opprettet et datasett kan vi bruke dette for å lage oss et corpus
å trene på/med.

Vi starter med å lage oss et sett med alt innhold, "corpus", og et sett med
"queries" (de elementene som vi ønsker å teste mot innholdet).

In [ ]:
# Merk at vi bruker `dataset` for å bruke _alt_ innhold
corpus = dict(dataset.select(["id", "positiv"]).rows())
# For "queries" bruker vi det vi har plukket ut i test
queries = dict(test_dataset.select(["id", "anchor"]).rows())

Vi trenger så å lage oss en mapping mellom "queries" og relevant innhold. I vårt
tilfellet så vil det være mer overlapp f.eks. fordi en tittel vil være relevant
for flere innholdbiter.

In [ ]:
relevant_docs = {}
for qid, q in queries.items():
    relevant_docs[qid] = dataset.filter(pl.col("anchor") == q).get_column("id").to_list()

## Språkmodell

Nå som vi har ordnet oss med litt data er det endelig på tide å velge en
språkmodell. Vi kommer til å bruke
[`sentence-transformers`](https://sbert.net/index.html) for modellen og trening
så la oss først ordne nødvendige pakker.

:::: {.callout-note}
## Nødvendige pakker

Vi trenger et par pakker for `sentence-transformers` og de avhenger av riktig
oppsett for effektiv trening.

::: {.panel-tabset}
## Uten CUDA (Linux, Mac og Windows uten GPU)
For maskiner uten dedikert Nvidia GPU kan man enkelt installere som følger:

```bash
uv add transformers --extra torch
uv add sentence-transformers --extra train
```

## CUDA

Hvis du har et dedikert grafikkort kan du tjene mye på å installere PyTorch med
CUDA støtte.

Her anbefaler vi å følge oppskriften på [PyTorch sin
hjemmeside](https://pytorch.org/get-started/locally/) for å få riktig oppsett
for akkurat din maskin.

Deretter trenger du:
```bash
uv add transformers
uv add sentence-transformers --extra train
```
:::

::::

---

Når det kommer til valg av språkmodell så er det vanskelig å gi noen konkrete
anbefalinger, nettopp fordi man kan tilpasse modellene til egne data slik vi
gjør her. En god oversikt over hvordan å velge språkmodell finnes i [Nav sin
tekniske
veileder](https://data.ansatt.nav.no/quarto/b9ec1385-d596-47e2-a3d2-8cbc85c577a3/_book/llm/lokale.html).

Vi kommer til å gå videre med
[`Alibaba-NLP/gte-modernbert-base`](https://huggingface.co/Alibaba-NLP/gte-modernbert-base).
Denne har vi valgt av følgende grunner:

- Den gjør det godt i sammenligninger mot embeddingmodeller av tilsvarende størrelse
- Det er en relativt liten, $149$ millioner parametere, modell som burde passe
fint på en laptop
- Den har et stort kontekstvindu på $8192$ token
- Den er lisensiert på en måte som gjør at vi enkelt kan ta den i bruk i Nav
(`Apache 2.0`)

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "Alibaba-NLP/gte-modernbert-base",
    # Merk at her velger vi `device` spesifikt tilpasset Mac
    # her burde man bruke "cuda" hvis tilgjengelig
    device="mps" if torch.backends.mps.is_available() else "cpu",
)

### Evaluere språkmodell

La oss nå se litt på hvordan modellen vår gjør det på datasettet vårt.

Vi må starte med å definere en måte å evaluere modellen vår, her har også
[`sentence-transformers` god
støtte](https://sbert.net/docs/sentence_transformer/training_overview.html#evaluator)
så vi benytter det som er innebygget der.

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers.util import cos_sim

evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="modernbert",
    score_functions={"cosine": cos_sim},
    batch_size=4,
    show_progress_bar=True,
)

Deretter kan vi benytte `evaluator` til å vurdere modellene vår.

In [ ]:
#| eval: false
base_eval = evaluator(model)
print(base_eval)